
# Multithreading in Python

This notebook covers various concepts and demonstrations for Multithreading in Python. Python provides threading capabilities through the `threading` module to perform concurrent execution of code. Each section includes explanations, sample code, and exercises for topics such as:

- Basics of Multithreading
- Creating Threads
- Thread Synchronization
- Lock Objects
- Thread Communication
- Using `ThreadPoolExecutor`
- Daemon Threads
- Thread Safety
- Global Interpreter Lock (GIL) and its impact


## Basics of Multithreading

In [7]:
# Example 1: Simple Multithreading Example
import threading

def print_numbers():
    for i in range(55):
        print(i)

# Create two threads
thread1 = threading.Thread(target=print_numbers)
thread2 = threading.Thread(target=print_numbers)

# Start both threads
thread1.start()
thread2.start()

# Wait for both threads to complete
thread1.join()
thread2.join()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54


## Creating Threads

In [21]:
# Example 2: Creating Threads with a Custom Class
import threading

class MyThread(threading.Thread):
    def run(self):
        for i in range(5):
            print(f"Thread {self.name} is running")

# Creating thread objects
thread1 = MyThread()
thread2 = MyThread()

# Start the threads
thread1.start()
thread2.start()

# Join the threads
thread1.join()
thread2.join()


Thread Thread-21 is running
Thread Thread-21 is running
Thread Thread-21 is running
Thread Thread-21 is running
Thread Thread-21 is running
Thread Thread-22 is running
Thread Thread-22 is running
Thread Thread-22 is running
Thread Thread-22 is running
Thread Thread-22 is running


## Thread Synchronization

In [17]:
a = 'python'

In [19]:
help(a.join)

Help on built-in function join:

join(iterable, /) method of builtins.str instance
    Concatenate any number of strings.

    The string whose method is called is inserted in between each given string.
    The result is returned as a new string.

    Example: '.'.join(['ab', 'pq', 'rs']) -> 'ab.pq.rs'



In [23]:
# Example 3: Thread Synchronization using Locks
import threading

lock = threading.Lock()
counter = 0

def increment_counter():
    global counter
    for _ in range(1000):
        with lock:
            counter += 1

# Creating two threads
thread1 = threading.Thread(target=increment_counter)
thread2 = threading.Thread(target=increment_counter)

# Start both threads
thread1.start()
thread2.start()

# Wait for both threads to finish
thread1.join()
thread2.join()

print(f"Final counter value: {counter}")


Final counter value: 2000


This code demonstrates **thread synchronization** using a `Lock` to safely update a shared variable (`counter`) from multiple threads. Without proper synchronization, simultaneous access to shared data from multiple threads can lead to race conditions, where the final value of the shared variable becomes unpredictable.

### Key Concepts:

1. **Shared Resource (`counter`)**:
   - `counter` is a global variable that is shared between the two threads (`thread1` and `thread2`).
   - Both threads are trying to increment the `counter` 1000 times in parallel.

2. **Race Condition**:
   - If two or more threads try to modify `counter` simultaneously without synchronization, the operations can interleave in ways that result in incorrect updates. This is called a **race condition**.

3. **Thread Synchronization using `Lock`**:
   - A `Lock` object is used to ensure that only one thread can modify the `counter` at a time, thus preventing race conditions.
   - A lock works like a switch: only one thread can "acquire" the lock, and all other threads have to wait until the lock is "released."

### Explanation of the Code:

```python
import threading

lock = threading.Lock()  # Create a lock object
counter = 0  # Shared resource (global variable)

def increment_counter():
    global counter
    for _ in range(1000):  # Each thread increments the counter 1000 times
        with lock:  # Acquire the lock before modifying the counter
            counter += 1  # Critical section: modifying the shared variable

# Creating two threads
thread1 = threading.Thread(target=increment_counter)
thread2 = threading.Thread(target=increment_counter)

# Start both threads
thread1.start()
thread2.start()

# Wait for both threads to finish
thread1.join()
thread2.join()

# Print the final value of the counter
print(f"Final counter value: {counter}")
```

### Step-by-Step Breakdown:

1. **Creating the Lock**: 
   ```python
   lock = threading.Lock()
   ```
   A `Lock` object is created. This lock will be used to control access to the shared resource `counter`.

2. **Defining the Thread Function**:
   ```python
   def increment_counter():
       global counter
       for _ in range(1000):
           with lock:  # Acquire the lock
               counter += 1  # Critical section (modifying the shared resource)
   ```
   - The function `increment_counter()` is designed to increment the `counter` variable 1000 times.
   - `with lock:` ensures that before each thread modifies `counter`, it first acquires the lock. Only one thread can acquire the lock at any given time, so the critical section (`counter += 1`) is executed by only one thread at a time.
   - Once a thread has completed modifying `counter`, it releases the lock, allowing the next thread to acquire it.

3. **Creating Threads**:
   ```python
   thread1 = threading.Thread(target=increment_counter)
   thread2 = threading.Thread(target=increment_counter)
   ```
   Two threads (`thread1` and `thread2`) are created, and both will execute the `increment_counter()` function.

4. **Starting the Threads**:
   ```python
   thread1.start()
   thread2.start()
   ```
   The two threads are started. At this point, they run concurrently, each trying to increment the `counter` 1000 times.

5. **Joining the Threads**:
   ```python
   thread1.join()
   thread2.join()
   ```
   The `join()` method ensures that the main program waits for both threads to complete their execution before moving on. Without this, the main thread might finish before the other threads complete, leading to incomplete results.

6. **Printing the Final Value**:
   ```python
   print(f"Final counter value: {counter}")
   ```
   After both threads have finished, the final value of `counter` is printed. Since both threads increment `counter` 1000 times each, the expected final value of `counter` should be `2000` if thread synchronization works correctly.

### Purpose of the `Lock`:
- Without the `lock`, there would be a race condition when both threads try to read and update `counter` simultaneously. Since reading and updating `counter` isn't an atomic operation, both threads could end up overwriting each other's changes, leading to incorrect results.
- The `lock` ensures that only one thread can modify `counter` at a time, thus guaranteeing correct behavior.

### Expected Output:
The final value of `counter` will be `2000` since both threads increment it by 1000, and the use of the `lock` ensures that there are no race conditions.

```python
Final counter value: 2000
```

### Conclusion:
This example demonstrates how to safely handle shared data between threads using a `Lock`. Without proper synchronization, such as using a `lock`, race conditions could cause unpredictable and incorrect behavior.

## Lock Objects

In [38]:
# Example 4: Using Lock to Protect Critical Sections
import threading

lock = threading.Lock()

def critical_section(thread_number):
    print(f"Thread {thread_number} attempting to enter critical section")
    with lock:
        print(f"Thread {thread_number} inside critical section")
    print(f"Thread {thread_number} exited critical section")

# Create and start multiple threads
threads = []
for i in range(5):
    t = threading.Thread(target=critical_section, args=(i,))
    threads.append(t)
    t.start()

for t in threads:
    t.join()


Thread 0 attempting to enter critical section
Thread 0 inside critical section
Thread 0 exited critical section
Thread 1 attempting to enter critical section
Thread 1 inside critical section
Thread 1 exited critical section
Thread 2 attempting to enter critical section
Thread 2 inside critical section
Thread 2 exited critical section
Thread 3 attempting to enter critical section
Thread 3 inside critical section
Thread 3 exited critical section
Thread 4 attempting to enter critical section
Thread 4 inside critical section
Thread 4 exited critical section


# Example 4: Using Lock to Protect Critical Sections
import threading

lock = threading.Lock()

def critical_section(thread_number):
    print(f"Thread {thread_number} attempting to enter critical section")
    with lock:
        print(f"Thread {thread_number} inside critical section")
    print(f"Thread {thread_number} exited critical section")

# Create and start multiple threads
threads = []
for i in range(5):
    t = threading.Thread(target=critical_section, args=(i,))
    threads.append(t)
    t.start()

for t in threads:
    t.join()


This code demonstrates how to use a `Lock` to control access to a **critical section** of code when multiple threads are running concurrently. A **critical section** is a part of a program where shared resources, such as variables, files, or memory, are accessed or modified. The `Lock` ensures that only one thread can enter the critical section at a time, avoiding race conditions.

### Key Concepts in the Code:

1. **Lock**:
   - A `Lock` is a synchronization primitive used to ensure that only one thread can execute a block of code (the **critical section**) at any given time. Other threads attempting to enter the critical section must wait until the lock is released.

2. **Thread**:
   - Multiple threads are created, each attempting to access the critical section. The lock ensures that only one thread can enter the critical section at a time.

### Code Breakdown:

```python
import threading

lock = threading.Lock()  # Create a lock object

def critical_section(thread_number):
    print(f"Thread {thread_number} attempting to enter critical section")
    with lock:  # Acquire the lock before entering the critical section
        print(f"Thread {thread_number} inside critical section")
    print(f"Thread {thread_number} exited critical section")
```

1. **Creating the Lock**: 
   - A `Lock` object is created using `threading.Lock()`. This lock will be used to synchronize the threads' access to the critical section.

2. **Defining the Critical Section**:
   - The function `critical_section(thread_number)` simulates a thread attempting to enter a critical section of the code.
   - The line `with lock:` is a context manager that automatically **acquires the lock** when a thread enters this block of code and **releases the lock** when the thread exits the block.
   - The lock ensures that only one thread can execute the code inside the `with lock:` block at any time.

3. **Thread Behavior**:
   - When a thread attempts to enter the critical section, it prints a message indicating its attempt.
   - If the thread acquires the lock, it prints another message indicating that it's inside the critical section.
   - After the critical section is completed, the lock is released, allowing other threads to enter.
   - Each thread prints a message when it exits the critical section.

### Thread Creation and Execution:

```python
# Create and start multiple threads
threads = []
for i in range(5):
    t = threading.Thread(target=critical_section, args=(i,))
    threads.append(t)
    t.start()
```

1. **Creating Multiple Threads**:
   - A loop is used to create 5 threads. Each thread runs the `critical_section()` function, passing its thread number (`i`) as an argument.

2. **Starting Threads**:
   - After each thread is created, it is started using `t.start()`. This causes the thread to begin execution concurrently with the others.

### Joining Threads:

```python
for t in threads:
    t.join()
```

- The `join()` method ensures that the main program waits for all the threads to finish their execution before continuing. Without `join()`, the main thread could finish and terminate the program before the other threads complete their tasks.

### How the Code Works:

1. **Thread Attempts**:
   - Each thread prints a message indicating that it is attempting to enter the critical section.
   
2. **Lock Behavior**:
   - Only one thread at a time can acquire the lock and enter the critical section. Other threads must wait until the current thread releases the lock.
   - Inside the critical section, the thread prints a message indicating it is inside.

3. **Exiting**:
   - Once the thread has finished its task inside the critical section, it exits, releasing the lock. At this point, another thread can acquire the lock and enter the critical section.

### Example Output:

The output will show that threads attempt to enter the critical section, and only one thread at a time can be inside the critical section due to the lock.

```plaintext
Thread 0 attempting to enter critical section
Thread 0 inside critical section
Thread 0 exited critical section
Thread 1 attempting to enter critical section
Thread 1 inside critical section
Thread 1 exited critical section
Thread 2 attempting to enter critical section
Thread 2 inside critical section
Thread 2 exited critical section
Thread 3 attempting to enter critical section
Thread 3 inside critical section
Thread 3 exited critical section
Thread 4 attempting to enter critical section
Thread 4 inside critical section
Thread 4 exited critical section
```

### Purpose of the Lock:

- The `lock` ensures **mutual exclusion**—that is, only one thread at a time can enter the critical section.
- Without the lock, multiple threads could simultaneously enter the critical section, potentially leading to **race conditions** or inconsistent behavior when accessing shared resources.

### Conclusion:

This example demonstrates the use of a **lock** to synchronize threads and protect a critical section. By using the lock, you ensure that multiple threads access shared resources in a safe and predictable manner, preventing race conditions.

## Thread Communication

In [40]:
# Example 5: Communication Between Threads Using a Queue
import threading
import queue

q = queue.Queue()

def producer():
    for i in range(5):
        print(f"Producing {i}")
        q.put(i)

def consumer():
    while not q.empty():
        item = q.get()
        print(f"Consuming {item}")

# Create producer and consumer threads
producer_thread = threading.Thread(target=producer)
consumer_thread = threading.Thread(target=consumer)

# Start threads
producer_thread.start()
producer_thread.join()

consumer_thread.start()
consumer_thread.join()


Producing 0
Producing 1
Producing 2
Producing 3
Producing 4
Consuming 0
Consuming 1
Consuming 2
Consuming 3
Consuming 4


This code demonstrates how to use a **queue** to facilitate communication between two threads: a **producer** thread and a **consumer** thread. In this example, the **producer** generates data and places it into the queue, while the **consumer** retrieves the data from the queue for processing.

### Key Concepts in the Code:
1. **Producer-Consumer Pattern**: 
   - The **producer** thread generates items (data) and places them into a queue.
   - The **consumer** thread retrieves and processes those items from the queue.
   - This pattern is a common synchronization model used in multithreaded programming to manage shared data between threads.

2. **Queue**:
   - A **queue** (from Python’s `queue` module) is used as a thread-safe data structure. It ensures proper synchronization when multiple threads are accessing it simultaneously.
   - The queue handles both locking and unlocking internally, so you don't need to use explicit locks.

### Code Breakdown:

```python
import threading
import queue

q = queue.Queue()  # Create a queue object
```

1. **Queue Initialization**: 
   - A `Queue` object `q` is created. This will act as the shared data structure where the producer can place items, and the consumer can retrieve them.
   - The `queue.Queue()` class provides built-in thread safety, so you don’t need to worry about race conditions when accessing the queue from multiple threads.

### Producer Function:

```python
def producer():
    for i in range(5):
        print(f"Producing {i}")
        q.put(i)  # Add items to the queue
```

- **Producer's Job**:
  - The `producer()` function generates 5 items (from 0 to 4).
  - For each item, it prints a message indicating what is being produced.
  - It then places the item into the queue using `q.put(i)`. The `put()` method safely adds an item to the queue.
  
### Consumer Function:

```python
def consumer():
    while not q.empty():
        item = q.get()  # Get an item from the queue
        print(f"Consuming {item}")
```

- **Consumer's Job**:
  - The `consumer()` function runs a loop that keeps consuming items from the queue until the queue is empty.
  - It uses `q.get()` to retrieve the next item from the queue. The `get()` method safely removes and returns an item from the queue.
  - The consumer then prints a message indicating what is being consumed.

### Thread Creation and Execution:

```python
# Create producer and consumer threads
producer_thread = threading.Thread(target=producer)
consumer_thread = threading.Thread(target=consumer)

# Start threads
producer_thread.start()
producer_thread.join()

consumer_thread.start()
consumer_thread.join()
```

1. **Creating Threads**:
   - Two threads are created: 
     - `producer_thread` runs the `producer()` function.
     - `consumer_thread` runs the `consumer()` function.

2. **Starting Threads**:
   - The `producer_thread` is started using `producer_thread.start()`, which begins executing the `producer()` function.
   - The `join()` method is called on `producer_thread`, making the main thread wait until the producer has finished producing all items.

3. **Starting the Consumer**:
   - After the producer has completed its work (verified by `producer_thread.join()`), the `consumer_thread` is started using `consumer_thread.start()`.
   - The `join()` method ensures the main thread waits until the consumer has finished consuming all items.

### How the Code Works:

1. **Producing Items**:
   - The producer generates 5 items (0 to 4) and places each item into the queue using `q.put(i)`. The `put()` method safely handles thread synchronization.

2. **Waiting for Producer to Finish**:
   - The main thread waits for the producer to finish producing items by calling `producer_thread.join()`. This ensures that the consumer only starts after the producer has finished its job.

3. **Consuming Items**:
   - The consumer starts after the producer has finished.
   - It retrieves and processes items from the queue using `q.get()`. It continues until the queue is empty.
   - The queue ensures that the consumer retrieves items in the same order they were added (FIFO order).

### Example Output:

```plaintext
Producing 0
Producing 1
Producing 2
Producing 3
Producing 4
Consuming 0
Consuming 1
Consuming 2
Consuming 3
Consuming 4
```

### Why `join()` is Important Here:
- **Producer First, Consumer After**: By using `producer_thread.join()`, you ensure that the producer completes its task before the consumer starts. This prevents the consumer from trying to retrieve items from an empty queue (before the producer has added any items).
- The `join()` method guarantees that the consumer doesn’t start until the producer has finished.

### Why Queue is Used:
- **Thread-Safe Communication**: The `queue.Queue()` provides thread-safe methods for adding (`put()`) and removing (`get()`) items. This allows both the producer and consumer to work with the same shared data structure (the queue) without needing explicit locks.
- The queue follows the **FIFO (First In, First Out)** principle, ensuring that items are consumed in the same order they were produced.

### Summary:

- **Producer-Consumer Pattern**: The producer adds items to the queue, and the consumer removes and processes them. This is a classic example of inter-thread communication.
- **Queue**: A thread-safe way to share data between threads without needing explicit locking mechanisms.
- **join()**: Ensures the threads execute in the desired order (producer finishes before the consumer starts).

This pattern is common in multithreading scenarios where different tasks need to be done by different threads while sharing data safely between them.

## Using `ThreadPoolExecutor`

In [42]:
# Example 6: Using ThreadPoolExecutor for Multithreading
from concurrent.futures import ThreadPoolExecutor

def task(n):
    print(f"Task {n} is being executed")

with ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(task, range(5))


Task 0 is being executed
Task 1 is being executed
Task 2 is being executed
Task 3 is being executed
Task 4 is being executed


In [46]:
# Example 6: Using ThreadPoolExecutor for Multithreading
from concurrent.futures import ThreadPoolExecutor
import threading

def task(n):
    worker_id = threading.get_ident()  # Get the unique identifier for the current worker (thread)
    print(f"Worker {worker_id} is executing Task {n}")

with ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(task, range(500))


Worker 23344 is executing Task 0
Worker 23344 is executing Task 1
Worker 23344 is executing Task 2
Worker 23344 is executing Task 3
Worker 23344 is executing Task 4
Worker 23344 is executing Task 5
Worker 23344 is executing Task 6
Worker 23344 is executing Task 7
Worker 23344 is executing Task 8
Worker 23344 is executing Task 9
Worker 23344 is executing Task 10
Worker 23344 is executing Task 11
Worker 23344 is executing Task 12
Worker 23344 is executing Task 13
Worker 23344 is executing Task 14
Worker 23344 is executing Task 15
Worker 23344 is executing Task 16
Worker 23344 is executing Task 17
Worker 23344 is executing Task 18
Worker 23344 is executing Task 19
Worker 23344 is executing Task 20
Worker 23344 is executing Task 21
Worker 23344 is executing Task 22
Worker 23344 is executing Task 23
Worker 23344 is executing Task 24
Worker 23344 is executing Task 25
Worker 23344 is executing Task 26
Worker 23344 is executing Task 27
Worker 23344 is executing Task 28
Worker 23344 is executin

This code demonstrates how to use the `ThreadPoolExecutor` from the `concurrent.futures` module to handle multithreading more efficiently. It provides a way to manage a pool of threads that can execute tasks concurrently without manually creating and starting individual threads.

### Key Concepts in the Code:

1. **Thread Pool**:
   - A **thread pool** is a collection of pre-instantiated, reusable threads. This pool manages a fixed number of threads that are available to execute tasks concurrently.
   - `ThreadPoolExecutor` helps simplify thread management by allowing you to execute multiple tasks in parallel without having to create, start, and manage threads manually.

2. **Executor**:
   - The `ThreadPoolExecutor` manages the pool of threads. You specify how many threads to have in the pool (`max_workers`) and the tasks that need to be executed. The executor automatically assigns tasks to available threads.

### Code Breakdown:

```python
from concurrent.futures import ThreadPoolExecutor

def task(n):
    print(f"Task {n} is being executed")
```

1. **Task Definition**:
   - The `task()` function takes an integer `n` as an argument and prints a message indicating that task `n` is being executed.
   - This function will be executed by different threads managed by the thread pool.

### Creating and Managing Threads Using `ThreadPoolExecutor`:

```python
with ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(task, range(5))
```

2. **ThreadPoolExecutor**:
   - The `ThreadPoolExecutor` is created with `max_workers=3`, meaning that at most **3 threads** will run concurrently.
   - The `with` statement ensures that the `executor` is properly cleaned up when all tasks are complete. It simplifies the process by automatically shutting down the executor when exiting the block.
   
3. **Mapping Tasks**:
   - `executor.map(task, range(5))` applies the `task` function to each value in `range(5)` (which is `[0, 1, 2, 3, 4]`).
   - Each value is passed to the `task()` function as the argument `n`. The executor assigns the execution of each task to one of the available threads in the pool.
   - Since `max_workers=3`, the first three tasks (0, 1, 2) will run concurrently. Once any of these threads finish, the remaining tasks (3, 4) will be assigned to the available threads.

### Execution Flow:

1. **Thread Pool Creation**:
   - A `ThreadPoolExecutor` with 3 threads is created, meaning up to 3 tasks can run at the same time.

2. **Task Assignment**:
   - The `map()` method assigns the `task()` function to each element of `range(5)`:
     - First, threads 0, 1, and 2 will start executing tasks 0, 1, and 2, respectively.
     - Once one of these threads completes, it will pick up tasks 3 and 4.
   
3. **Automatic Thread Management**:
   - The `ThreadPoolExecutor` automatically handles task assignment and thread reuse.
   - After all tasks are completed, the thread pool is automatically shut down (because of the `with` statement).

### Example Output:

```
Task 0 is being executed
Task 1 is being executed
Task 2 is being executed
Task 3 is being executed
Task 4 is being executed
```

- The order of the output may vary because tasks 0, 1, and 2 are executed concurrently, followed by tasks 3 and 4. You might see output in different sequences depending on thread scheduling, but all 5 tasks will be executed.

### Benefits of Using `ThreadPoolExecutor`:

1. **Efficient Thread Management**:
   - Instead of manually creating and managing threads, `ThreadPoolExecutor` manages a pool of threads for you, reusing threads to handle tasks efficiently.

2. **Concurrency Control**:
   - You can control how many threads run in parallel by adjusting the `max_workers` argument. This can help manage system resources, especially when dealing with a large number of tasks.

3. **Automatic Cleanup**:
   - Using `ThreadPoolExecutor` inside a `with` statement ensures that threads are cleaned up automatically once all tasks are completed, making the code cleaner and less error-prone.

4. **Task Submission**:
   - The `map()` function allows you to submit multiple tasks at once, mapping each input to the corresponding function, simplifying the process of assigning work to threads.

### Summary:

This example illustrates how `ThreadPoolExecutor` can be used to execute multiple tasks concurrently using a thread pool, simplifying the process of managing threads and improving resource management. It’s ideal when you have multiple tasks to run in parallel and want to efficiently manage the available threads.

## Daemon Threads

In [ ]:
# Example 7: Daemon Threads
import threading
import time

def daemon_thread():
    while True:
        print("Daemon thread is running")
        time.sleep(1)

# Creating a daemon thread
thread = threading.Thread(target=daemon_thread)
thread.setDaemon(True)

# Start the daemon thread
thread.start()

# Main thread sleeping for 3 seconds
time.sleep(3)
print("Main thread exiting")


This code demonstrates the concept of **daemon threads** in Python, which are threads that run in the background and do not prevent the program from exiting. Once all non-daemon threads (main thread or other non-daemon threads) complete, the program will exit, even if daemon threads are still running.

### Key Concepts in the Code:

1. **Daemon Threads**:
   - **Daemon threads** are background threads that run continuously but do not block the program from exiting. When the main thread (or other non-daemon threads) finishes execution, any running daemon threads are **abruptly terminated** without completing their current tasks.
   - Daemon threads are used for tasks that should run in the background (like monitoring services), but it’s okay if they are stopped when the main program exits.

2. **Main Thread**:
   - The main thread is the default thread of execution in a Python program. It’s the thread that starts and finishes the program, and by default, all threads are non-daemon unless explicitly set as daemon threads.

### Code Breakdown:

```python
import threading
import time

def daemon_thread():
    while True:
        print("Daemon thread is running")
        time.sleep(1)
```

1. **Daemon Thread Function**:
   - The `daemon_thread()` function contains an infinite loop that prints "Daemon thread is running" every second. This loop will run continuously as long as the thread is active, but it doesn't have a defined stopping point because it’s designed to simulate background work.

### Creating and Setting a Daemon Thread:

```python
# Creating a daemon thread
thread = threading.Thread(target=daemon_thread)
thread.setDaemon(True)  # Set the thread as a daemon thread
```

2. **Thread Creation**:
   - A thread `thread` is created that will execute the `daemon_thread()` function when started.
   - **Setting the Daemon Flag**:
     - `thread.setDaemon(True)` marks this thread as a daemon thread. This means the thread will run in the background and will be abruptly terminated when the main thread finishes its execution, even if it’s in the middle of its work.

### Starting and Running the Threads:

```python
# Start the daemon thread
thread.start()

# Main thread sleeping for 3 seconds
time.sleep(3)
print("Main thread exiting")
```

3. **Starting the Daemon Thread**:
   - The `thread.start()` method starts the daemon thread, which begins executing the `daemon_thread()` function.
   - The daemon thread runs concurrently with the main thread.

4. **Main Thread Execution**:
   - The main thread (the default thread running the program) pauses for 3 seconds (`time.sleep(3)`), allowing the daemon thread to run during this time.
   - After the 3 seconds, the main thread prints "Main thread exiting" and exits the program.

### Behavior of Daemon Threads:

- While the main thread is sleeping, the daemon thread is actively running and printing "Daemon thread is running" every second. However, since the daemon thread is running in the background, once the main thread exits (after 3 seconds), the program terminates, **abruptly stopping the daemon thread**.
- Even though the daemon thread is in an infinite loop, it will be killed as soon as the main thread completes.

### Output Example:

```plaintext
Daemon thread is running
Daemon thread is running
Daemon thread is running
Main thread exiting
```

- The daemon thread runs and prints its message every second. After 3 seconds, the main thread exits, and the daemon thread is terminated without further output.

### Important Points:

1. **Daemon vs. Non-Daemon Threads**:
   - **Non-daemon threads**: The main program will wait for all non-daemon threads to finish before it exits.
   - **Daemon threads**: The program will not wait for daemon threads to finish, and they are killed once all non-daemon threads (including the main thread) are done.

2. **Usage of Daemon Threads**:
   - Daemon threads are typically used for tasks that should run in the background, like monitoring, logging, or handling events, but the program does not need to wait for them to finish.

3. **Termination of Daemon Threads**:
   - Daemon threads are terminated as soon as the main thread finishes, even if they are in the middle of an operation. This means they do not complete their work gracefully, so daemon threads should not be used for critical tasks.

### Summary:

- In this example, a daemon thread runs concurrently with the main thread. The daemon thread prints a message every second. After 3 seconds, the main thread exits, causing the daemon thread to terminate immediately, even though the daemon thread is in an infinite loop.
- **Daemon threads** are ideal for background operations that don't require a clean shutdown, but they should not be used for tasks that must be completed before the program exits.

## Thread Safety

In [ ]:
# Example 8: Thread Safety with Local Variables
import threading

def safe_task():
    local_data = threading.local()
    local_data.value = 1
    for _ in range(5):
        local_data.value += 1
        print(f"Thread {threading.current_thread().name} - Value: {local_data.value}")

# Creating threads
thread1 = threading.Thread(target=safe_task)
thread2 = threading.Thread(target=safe_task)

# Start and join threads
thread1.start()
thread2.start()

thread1.join()
thread2.join()


This code demonstrates how to achieve **thread safety** using **thread-local storage** in Python. The code utilizes the `threading.local()` mechanism to provide each thread with its own copy of a variable, ensuring that there is no interference between threads even though they are executing the same task.

### Key Concepts in the Code:

1. **Thread Safety**:
   - **Thread safety** refers to the concept of ensuring that multiple threads can access shared data or resources concurrently without causing data corruption or inconsistencies.
   - In this example, each thread has its own local data, so there's no need for explicit synchronization (like `Lock` objects) because each thread works independently on its own data.

2. **Thread-Local Storage**:
   - The `threading.local()` object creates **thread-local storage**, which provides each thread with its own independent instance of variables. This means that even though all threads access the same object, they get their own isolated copies of the data stored in that object.
   - This prevents the threads from interfering with each other's data, allowing them to operate safely in parallel.

### Code Breakdown:

```python
import threading

def safe_task():
    local_data = threading.local()  # Create a thread-local storage object
    local_data.value = 1  # Each thread will have its own 'value' variable
    for _ in range(5):
        local_data.value += 1  # Increment the value in a loop
        print(f"Thread {threading.current_thread().name} - Value: {local_data.value}")
```

1. **Thread-Local Storage Object**:
   - `local_data = threading.local()` creates a **thread-local storage** object. Each thread gets its own independent `local_data` object when executing the `safe_task()` function.
   - Inside the thread, the variable `local_data.value` is initialized to 1 for each thread. This means that each thread has its own independent copy of `local_data.value` to work with.

2. **Thread Behavior**:
   - Each thread increments `local_data.value` five times in a loop. The `print()` statement shows the current thread’s name and the value of `local_data.value` for that specific thread.
   - Since each thread has its own `local_data`, there is no risk of one thread modifying another thread’s data.

### Creating and Starting Threads:

```python
# Creating threads
thread1 = threading.Thread(target=safe_task)
thread2 = threading.Thread(target=safe_task)

# Start and join threads
thread1.start()
thread2.start()

thread1.join()
thread2.join()
```

3. **Thread Creation**:
   - Two threads (`thread1` and `thread2`) are created. Both threads run the `safe_task()` function.
   - Even though both threads are executing the same function, each thread will have its own independent copy of the `local_data` variable, ensuring thread safety.

4. **Starting Threads**:
   - `thread1.start()` and `thread2.start()` start the threads, allowing them to run concurrently.
   - The threads will each execute `safe_task()` in parallel.

5. **Joining Threads**:
   - `thread1.join()` and `thread2.join()` ensure that the main thread waits for both threads to complete their tasks before exiting the program.

### How the Code Works:

1. **Thread-Local Storage**:
   - When each thread runs `safe_task()`, it creates its own version of `local_data.value`. Even though both threads use the same `safe_task()` function, their `local_data` is isolated, and changes made by one thread do not affect the other.

2. **Thread Execution**:
   - Both threads increment their own copy of `local_data.value` from 1 to 6 (each thread increments the value five times).
   - The `print()` statements show that both threads operate independently on their own copy of `local_data.value`, with no conflicts.

### Example Output:

You might see an output like this:

```plaintext
Thread Thread-1 - Value: 2
Thread Thread-1 - Value: 3
Thread Thread-1 - Value: 4
Thread Thread-1 - Value: 5
Thread Thread-1 - Value: 6
Thread Thread-2 - Value: 2
Thread Thread-2 - Value: 3
Thread Thread-2 - Value: 4
Thread Thread-2 - Value: 5
Thread Thread-2 - Value: 6
```

In this example:
- `Thread-1` and `Thread-2` are both executing the same function, but their values remain independent.
- Each thread increments its `local_data.value` independently from 1 to 6.

### Important Points:

1. **Thread-Local Data**:
   - The key concept here is that each thread gets its own instance of the `local_data` object. Even though both threads use the same function, the data inside `local_data` is not shared between them.
   - This makes the code **thread-safe**, as there’s no risk of one thread interfering with another thread's data.

2. **Thread Safety without Locks**:
   - Normally, when multiple threads access shared data, you need to use synchronization primitives (like `Lock`) to avoid race conditions.
   - However, in this example, there’s no need for synchronization because each thread operates on its own private copy of the data using thread-local storage.

3. **Concurrency**:
   - The code allows multiple threads to run concurrently without conflicting over shared resources, which is a critical aspect of writing safe multithreaded applications.

### Summary:

This example shows how to achieve **thread safety** using **thread-local storage** (`threading.local()`), where each thread gets its own independent version of a variable. This prevents data conflicts without needing locks or other synchronization mechanisms. It's an efficient and easy-to-use solution for scenarios where threads need to work with independent copies of the same data.

## Global Interpreter Lock (GIL)

In [ ]:
# Example 9: Impact of Global Interpreter Lock (GIL)
import threading
import time

def cpu_bound_task():
    start = time.time()
    count = 0
    while time.time() - start < 2:
        count += 1
    print(f"Thread {threading.current_thread().name} completed")

# Creating threads to simulate CPU-bound tasks
thread1 = threading.Thread(target=cpu_bound_task)
thread2 = threading.Thread(target=cpu_bound_task)

# Start and join threads
thread1.start()
thread2.start()

thread1.join()
thread2.join()


This code demonstrates the effect of Python's **Global Interpreter Lock (GIL)** in the context of **CPU-bound tasks** in a multithreaded environment. It creates two threads that perform CPU-intensive operations (incrementing a counter) for two seconds and runs them concurrently. The output shows how Python handles multithreading for CPU-bound tasks due to the GIL.

### Key Concepts:

1. **Global Interpreter Lock (GIL)**:
   - The **GIL** is a mutex (lock) in Python that allows only one thread to execute Python bytecode at a time. Even in a multithreaded program, only one thread can run Python code at once.
   - The GIL is in place to protect memory management in CPython (the reference implementation of Python), ensuring thread safety for internal data structures.
   - The GIL is especially noticeable in **CPU-bound tasks**, where threads compete for CPU resources, causing them to run serially instead of concurrently.

2. **CPU-Bound Tasks**:
   - A **CPU-bound task** is a task that primarily uses the CPU for computation (e.g., complex calculations or incrementing a counter in a loop). These tasks are limited by the CPU’s speed, not by I/O operations (like reading from a file or network).
   - In this code, the threads are performing a CPU-bound task: repeatedly incrementing a counter for 2 seconds.

### Code Breakdown:

```python
import threading
import time

def cpu_bound_task():
    start = time.time()
    count = 0
    while time.time() - start < 2:  # Run for 2 seconds
        count += 1  # Perform a CPU-bound task: incrementing a counter
    print(f"Thread {threading.current_thread().name} completed")
```

1. **CPU-Bound Task**:
   - The function `cpu_bound_task()` simulates a CPU-bound task. It increments a counter (`count`) in a loop for 2 seconds.
   - `while time.time() - start < 2` ensures that the loop runs for 2 seconds.
   - The task is computationally intensive because it is continuously incrementing the counter.

### Thread Creation and Execution:

```python
# Creating threads to simulate CPU-bound tasks
thread1 = threading.Thread(target=cpu_bound_task)
thread2 = threading.Thread(target=cpu_bound_task)

# Start and join threads
thread1.start()
thread2.start()

thread1.join()
thread2.join()
```

2. **Thread Creation**:
   - Two threads (`thread1` and `thread2`) are created, both executing the `cpu_bound_task()` function. This simulates two CPU-bound tasks running concurrently.

3. **Starting Threads**:
   - The threads are started using `thread1.start()` and `thread2.start()`, causing them to run concurrently.
   - Each thread will attempt to execute `cpu_bound_task()` for 2 seconds.

4. **Joining Threads**:
   - The `join()` calls ensure that the main thread waits for both `thread1` and `thread2` to finish before proceeding. This ensures that the threads have completed their CPU-bound tasks before the program exits.

### Expected Behavior (with GIL):

Due to the **Global Interpreter Lock (GIL)**, even though two threads are created and executed concurrently, the GIL prevents true parallel execution of Python bytecode in CPU-bound tasks. Here's why:

1. **Thread Scheduling**:
   - Python switches between threads, allowing only one thread to execute at a time. Even though both threads are ready to run, the GIL ensures that only one of them can run Python bytecode at any moment. 
   - The GIL is released and reacquired periodically, allowing the second thread to execute. However, this switch happens frequently, giving the illusion of concurrency, but in reality, the threads are running in a serialized fashion on a single CPU core.

2. **Performance Impact**:
   - Since the GIL restricts parallel execution for CPU-bound tasks, the performance benefit of using multiple threads for CPU-intensive operations in Python is limited. Both threads take turns executing, reducing overall efficiency when compared to true parallel execution.

### Example Output:

The output will show that each thread completes its task, but due to the GIL, they are not truly running in parallel. Instead, the threads alternate their execution:

```plaintext
Thread Thread-1 completed
Thread Thread-2 completed
```

Even though both threads are performing CPU-bound tasks for 2 seconds, the GIL ensures that only one thread can run at a time. The program will finish after roughly 2 seconds, as both threads take turns running for those 2 seconds.

### Why the GIL Affects CPU-Bound Tasks:

1. **GIL and Python Threads**:
   - In Python, the GIL prevents multiple threads from executing Python bytecode simultaneously, which means that CPU-bound tasks in multiple threads do not benefit from multiple CPU cores.
   - Python's GIL ensures that only one thread executes Python bytecode at a time, regardless of how many threads you create.

2. **Multithreading in Python**:
   - Multithreading in Python is more suitable for **I/O-bound tasks** (e.g., file operations, network requests), where threads can wait for I/O to complete, allowing other threads to run while waiting. In such cases, the GIL is less of an issue because threads spend much of their time waiting.

3. **For CPU-Bound Tasks, Use Multiprocessing**:
   - If you want to take advantage of multiple CPU cores for CPU-bound tasks, you should use the **`multiprocessing` module** instead of `threading`. The `multiprocessing` module spawns separate processes, each with its own Python interpreter and GIL, allowing true parallelism across CPU cores.

### Summary:

- This example demonstrates how the **Global Interpreter Lock (GIL)** in Python affects **CPU-bound tasks** in a multithreaded environment.
- Although two threads are created and started concurrently, the GIL ensures that only one thread runs at a time, leading to serialized execution for CPU-bound operations.
- Python's `threading` is more effective for **I/O-bound tasks** where the GIL has minimal impact. For CPU-bound tasks requiring parallelism, the `multiprocessing` module is a better option.

## Thread Synchronization using Event

In [ ]:
# Example 10: Using Event for Thread Synchronization
import threading
import time

event = threading.Event()

def task():
    print("Waiting for event to be set")
    event.wait()
    print("Event received, task completed")

# Create and start the thread
thread = threading.Thread(target=task)
thread.start()

time.sleep(2)
print("Setting the event")
event.set()

thread.join()


This code demonstrates the use of an **Event** object in Python for thread synchronization. The **Event** object allows one or more threads to wait for an event to occur, and it provides a mechanism for communication between threads. In this case, the event acts as a signal that controls when a thread proceeds with its execution.

### Key Concepts:

1. **Event Object**:
   - An **Event** is a synchronization primitive provided by the `threading` module. It allows threads to wait for an event to be set before continuing their execution.
   - An event starts in a "cleared" state, meaning that it is not set. When another thread calls `set()` on the event, the event transitions to a "set" state, allowing waiting threads to proceed.

2. **Thread Synchronization**:
   - The `event.wait()` method makes the thread pause its execution until the event is set. This ensures that the thread doesn't proceed until it receives the signal.

### Code Breakdown:

```python
import threading
import time

event = threading.Event()  # Create an event object
```

1. **Creating the Event**:
   - The `event = threading.Event()` creates an event object. Initially, the event is in the "cleared" state, meaning threads waiting for the event will block until it is set.

### Task Function:

```python
def task():
    print("Waiting for event to be set")
    event.wait()  # Block here until the event is set
    print("Event received, task completed")
```

2. **Task Function**:
   - The `task()` function represents the work that a thread will perform.
   - When `event.wait()` is called, the thread pauses and waits until the event is set. At this point, the thread is blocked.
   - Once the event is set (by calling `event.set()`), the thread unblocks and prints "Event received, task completed."

### Thread Creation and Execution:

```python
# Create and start the thread
thread = threading.Thread(target=task)
thread.start()

time.sleep(2)
print("Setting the event")
event.set()  # Set the event to unblock the waiting thread
```

3. **Thread Creation**:
   - A thread is created using `threading.Thread(target=task)`, meaning the `task()` function will be executed by this thread.
   - `thread.start()` starts the thread, which begins by printing "Waiting for event to be set" and then pauses at `event.wait()`.

4. **Main Thread Control**:
   - The main thread sleeps for 2 seconds using `time.sleep(2)`. During this time, the thread running `task()` is blocked, waiting for the event to be set.
   - After the 2 seconds, the main thread calls `event.set()`, which signals the waiting thread that the event has been set. This unblocks the waiting thread and allows it to proceed.

### Synchronization Mechanism:

```python
event.set()
```

5. **Setting the Event**:
   - `event.set()` changes the state of the event to "set," signaling that the waiting thread can now proceed.
   - Any thread waiting on the event with `event.wait()` will be unblocked and allowed to continue execution.

### Waiting for the Thread to Finish:

```python
thread.join()
```

6. **Thread Completion**:
   - The `thread.join()` method ensures that the main thread waits for the child thread (the one executing `task()`) to finish its execution before exiting the program. This guarantees that the program doesn't terminate prematurely before the task is completed.

### Output Explanation:

1. Initially, the thread running `task()` prints "Waiting for event to be set" and then waits at `event.wait()`.
2. After a 2-second delay, the main thread sets the event with `event.set()`, causing the waiting thread to unblock.
3. The thread prints "Event received, task completed" and then finishes execution.

### Example Output:

```plaintext
Waiting for event to be set
Setting the event
Event received, task completed
```

- The `task()` thread prints "Waiting for event to be set" and blocks at `event.wait()`.
- After 2 seconds, the main thread prints "Setting the event" and sets the event, allowing the `task()` thread to proceed.
- The `task()` thread then prints "Event received, task completed" and finishes.

### Summary:

- **Event Object**: The `threading.Event` object is used for communication between threads, allowing one thread to signal another that an event has occurred. It provides a simple way to synchronize threads.
- **Thread Synchronization**: The thread running `task()` waits until the event is set before proceeding, ensuring proper synchronization between the main thread and the worker thread.
- **Real-World Use Case**: This pattern is useful in scenarios where one thread needs to wait for another thread to finish a specific task or where threads need to be coordinated to perform work in a specific order.

In this example, the event object acts as a trigger to control when the `task()` thread proceeds with its execution.